This notebook was created to support the data preparation required to support our CS 598 DLH project.  The paper we have chosen for the reproducibility project is:
***Ensembling Classical Machine Learning and Deep Learning Approaches for Morbidity Identification from Clinical Notes ***

This notebook is for creating the multiple embeddings formats as described in the study.

 

The data cannot be shared publicly due to the agreements required to obtain the data so we are storing the data locally and not putting in GitHub.

We are only creating embeddings for data that includes stop words.

In [72]:
import pandas as pd
import numpy as np

DATA_PATH = './obesity_data/'
test_df = pd.read_pickle(DATA_PATH + '/test_df.pkl')
train_df = pd.read_pickle(DATA_PATH + '/train_df.pkl')

test_df['word_count'] = test_df['lower_text'].apply(lambda x: len(x.split()))
train_df['word_count'] = train_df['lower_text'].apply(lambda x: len(x.split()))

df_print = pd.DataFrame()
df_print['Min'] = [np.min(test_df['word_count']), np.min(train_df['word_count'])]
df_print['Mean'] = [np.mean(test_df['word_count']), np.mean(train_df['word_count'])]
df_print['Max'] = [np.max(test_df['word_count']), np.max(train_df['word_count'])]
df_print['Std'] = [np.std(test_df['word_count']), np.std(train_df['word_count'])]
df_print['MeanPlusStd'] = round(df_print['Mean'] + df_print['Std'],0)
token_max = int(round(np.max(df_print['MeanPlusStd']),0))

print(df_print)
print('Max Tokens:',token_max)
print('Test:', sum(test_df['word_count'] > token_max), "out of", len(test_df))
print('Train:', sum(train_df['word_count'] > token_max), "out of", len(train_df))


   Min        Mean   Max         Std  MeanPlusStd
0  206  991.907298  3124  437.621770       1430.0
1  113  958.774141  3748  444.819254       1404.0
Max Tokens: 1430
Test: 79 out of 507
Train: 72 out of 611


We are going to split these larger text blocks into 2 notes of size max_token or below.  Note, there are 4 notes (1 in test and 3 in train) that are bigger than 2 times x tokens.  For now, we will ignore, but may want to add in later (either loop or have left/middle/right).

In [73]:
test_df_ok = test_df[test_df['word_count'] <= token_max].copy()
test_df_large_right = test_df[test_df['word_count'] > token_max].copy()
test_df_large_left = test_df[test_df['word_count'] > token_max].copy()

train_df_ok = train_df[train_df['word_count'] <= token_max].copy()
train_df_large_right = train_df[train_df['word_count'] > token_max].copy()
train_df_large_left = train_df[train_df['word_count'] > token_max].copy()

#Get the right words and the left words and then concatenate all 3 and recacluate 
test_df_large_left['lower_text'] = test_df_large_left['lower_text'].apply(lambda x: ' '.join([word for word in x.split()[:(token_max-1)]]))
test_df_large_right['lower_text'] = test_df_large_right['lower_text'].apply(lambda x: ' '.join([word for word in x.split()[token_max:(2*token_max)]]))
train_df_large_left['lower_text'] = train_df_large_left['lower_text'].apply(lambda x: ' '.join([word for word in x.split()[:(token_max-1)]]))
train_df_large_right['lower_text'] = train_df_large_right['lower_text'].apply(lambda x: ' '.join([word for word in x.split()[token_max:(2*token_max)]]))

test_df_expanded = pd.concat([test_df_ok,test_df_large_right,test_df_large_left])
test_df_expanded['word_count'] = test_df_expanded['lower_text'].apply(lambda x: len(x.split()))
train_df_expanded = pd.concat([train_df_ok,train_df_large_right,train_df_large_left])
train_df_expanded['word_count'] = train_df_expanded['lower_text'].apply(lambda x: len(x.split()))

print('Test:', sum(test_df_expanded['word_count'] > token_max), "out of", len(test_df_expanded))
print('Train:', sum(train_df_expanded['word_count'] > token_max), "out of", len(train_df_expanded))

Test: 0 out of 586
Train: 0 out of 683


![Note occurrences](images\note_occurrences.gif)
